In [1]:
!pip install pyspellchecker
!pip install -U transformers sklearn torchtext
from sklearn.svm import LinearSVC
import pandas as pd
from sklearn.pipeline import Pipeline
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import nltk
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker
from sklearn.metrics import classification_report
import numpy as np
from google.colab import files
from google.colab import drive
from torchtext.vocab import GloVe
import pickle
from sklearn.model_selection import GridSearchCV
from fastai import *
from fastai.text import *
from pathlib import Path
import pandas as pd
import numpy as np
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import transformers as tfms
import torch
from tqdm import tqdm_notebook as tn
import pickle
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import warnings
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
warnings.filterwarnings("ignore")

     |████████████████████████████████| 1.9MB 4.9MB/s 
     |████████████████████████████████| 389kB 4.7MB/s 
Requirement already up-to-date: sklearn in /usr/local/lib/python3.6/dist-packages (0.0)
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 860kB 50.4MB/s 
     |████████████████████████████████| 675kB 47.6MB/s 
     |████████████████████████████████| 1.0MB 51.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=369dd08a29ab552b06898f4ade38d038df900a300d6954c69b4848dbd80ceecd
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609175 sha256=5a1ed1720dee3f9998b213c889e8520545f06da9b6d74816981e10cd10f78d25
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
Successfully built sacremoses regex
  Fou

In [2]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
cd '/content/gdrive/My Drive/Colab Notebooks/NLP-Basic-to-Bert'

/content/gdrive/My Drive/Colab Notebooks/NLP-Basic-to-Bert


In [0]:
# Load the dataset
train_df = pd.read_csv('training_data.csv')
test_df = pd.read_csv('testing_data.csv')

# Find the number of words in every review
train_df['length']= train_df.user_review.str.split().apply(len)
test_df['length'] = test_df.user_review.str.split().apply(len)

# Remove the empty reviews
train_df= train_df[train_df.length!=0]
test_df = test_df[test_df.length!=0]

In [6]:
# Download the model, list of tokens and corresponding pretraining class
model_class, tokenizer_class, pretrained_weights = (tfms.BertModel, tfms.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [0]:
# Load the sample of the dataset as per resource constraints

#sample_df = train_df.head(1000)
#sample_test_df = test_df.head(1000)

sample_df = train_df 
sample_test_df = test_df

In [0]:
# Convert the words into bert tokens
tokenized = sample_df['user_review'].str.lower().apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512)))
test_tokenized = sample_test_df['user_review'].str.lower().apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512)))

In [0]:
# Padd zero at the end 
max_len = max(tokenized.apply(len)) 
max_test = max(test_tokenized.apply(len))
if max_test>max_len:
  max_len = max_test
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
padded_test = np.array([i + [0]*(max_len-len(i)) for i in test_tokenized.values])

In [0]:
# Add attention mask where zero is padded
attention_mask = np.where(padded != 0, 1, 0)
attention_mask_test = np.where(padded_test != 0, 1, 0)

In [0]:
# Convert pad and attention mask to tensor format
padded = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

In [0]:
padded_test = torch.tensor(padded_test)
attention_mask_test = torch.tensor(attention_mask_test)

In [0]:
# Dump the padded and attention for future use
pickle.dump((padded, attention_mask), open('bert_pad_attention', 'wb'))
pickle.dump((padded_test, attention_mask_test), open('bert_pad_attention_test', 'wb'))

In [0]:
# Create the dataloader for the train and test dataset
train_ds = TensorDataset(padded, attention_mask, torch.tensor(sample_df['user_rating'].values))
train_dl = DataLoader(train_ds, batch_size=256, shuffle=True)
test_ds = TensorDataset(padded_test, attention_mask_test, torch.tensor(sample_test_df['user_rating'].values))
test_dl = DataLoader(test_ds, batch_size=256, shuffle=True)

In [22]:
# Get the last hidden states as the features of the user review
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')
model = model.to(device)
ftrs = []
train_class = []
for i, m, y in tn(train_dl):
    input_ids = i.to(device)
    attention_mask = m.to(device)
    with torch.no_grad():
        last_hidden_states = model(input_ids, attention_mask=attention_mask)[0]
        ftrs.append(last_hidden_states[:,0,:].cpu().numpy())
        train_class.extend(y.numpy().tolist())

In [25]:

features = np.concatenate(ftrs)
len(train_class),len(ftrs),len(features)

(499999, 1954, 499999)

In [26]:
# Get the last hidden states as the features of the test data
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')
model = model.to(device)
ftrs_test = []
test_class = []
for i, m, y in tn(test_dl):
    input_ids = i.to(device)
    attention_mask_test = m.to(device)
    with torch.no_grad():
        last_hidden_states = model(input_ids, attention_mask=attention_mask_test)[0]
        ftrs_test.append(last_hidden_states[:,0,:].cpu().numpy())
        test_class.extend(y.numpy().tolist())

In [0]:
# Dump the bert features for future usage
features = np.concatenate(ftrs)
np.save('bert_features.npy', features)

features_test = np.concatenate(ftrs_test)
np.save('bert_features_test.npy', features_test)
np.save('bert_labels.npy',train_class)
np.save('bert_labels_test.npy',test_class)

In [0]:
# Find the best parameters using Grid Search
#model =LinearSVC()
#parameters = {'C': [0.01,0.1,1,10,100]}
#grid_search = GridSearchCV(LinearSVC(), parameters, n_jobs=-1)
#grid_search.fit(features, labels)
#print('best parameters: ', grid_search.best_params_)

In [2]:
# Create the model with best parameters
lr_clf = LinearSVC(C=10)
lr_clf.fit(features, train_class)

NameError: ignored

In [0]:
prediction=lr_clf.predict(features_test)

In [0]:
print(classification_report(test_class, prediction))

In [0]:
lr_clf = LinearSVC(C=100)
lr_clf.fit(features, train_class)
prediction=lr_clf.predict(features_test)
print(classification_report(test_class, prediction))

In [1]:
lr_clf = LinearSVC(C=0.1)
lr_clf.fit(features, train_class)
prediction=lr_clf.predict(features_test)
print(classification_report(test_class, prediction))

NameError: ignored

In [0]:
lr_clf = LinearSVC(C=0.01)
lr_clf.fit(features, train_class)
prediction=lr_clf.predict(features_test)
print(classification_report(test_class, prediction))